# YuleSimonProcess(alpha,Ts,initial_Tagset,new_tagset,how_new_tag)<br>
<b>alpha</b>: float, list of float, list of int:default:0.1<br>
もしfloatならタグを選ぶときにランダムに0~1を生成し、その値がalpha以下なら新規タグを生成する。もしfloatが入っているリストなら、それぞれのタイムステップごとにその確率で新規タグを生成する。もしintのリストなら、それぞれのタイムステップでその数分タグを生成する。<br>
・<b>Ts</b>:
(int,int)、list of int:default:(1,100)<br>
もし　intのタプルの場合、Ts[0]を1投稿におけるタグの生成数、Ts[1]を投稿数として生成する。
もしintのlistの場合、それぞれのタイムステップごとにその数分生成する。
<br>
・<b>initial_Tagset</b>:
list of list :default:[[0]]<br>
[[0,1],[0,0]]のような形でリストのリストで指定。
初期状態となる。
<br>
・<b>new_tagset</b>:
str, list :default:""<br>
strの場合、how_new_tagに従う。
listの場合、もし新規タグが出現するとき、そのlistの中身が順番で出現する。
もし出し尽くしたら、how_new_tagに従う。
<br>
・<b>how_new_tag</b>:
"enumerate","str_enumerate" and other str :default:"enumerate"<br>
もし"enumerate"だと新規タグを数値で生成する(1,2,3....)。
もし"str_enumerate"だと新規タグを数値の文字列で生成する("1","2","3"....)。
もしそれ以外の場合生成しない。<br>
・<b>fixed_T</b>:
bool:default:False<br>
もし、Trueの場合,タグに被りを許さず、再生成する。Falseの場合は許す。<br>

In [4]:
def YuleSimonProcess(alpha=0.1,Ts=(1,100),initial_Tagset=[[0]],new_tagset="",how_new_tag="enumerate",fixed_T=False):
    import numpy as np
    import random 
    create_flag=0
    from_num_new_tag=1
    #alpha:float of novelty rate or list of each post's novelty rate or function of novelty rate of each timestep[1..N].:default=0.03
    #Ts:(int,int) or list of number of Tag each post(and number of post).Default=(1,100)
    #initial_Tagset: list of list of new_tag:default: [[0]]
    #new_tagset:str or list of new tag set,if str or there is no non-used new-tag, followed how_new_tag: default:na
    #how_new_tag:"enumerate","str_enumerate" or else: if enumerate, this algorith give order number else, stop making a new_tag.
    #from_num_new_tag:int of starting point of enumerate of new tag :default:from_num_new_tag=0
    if type(initial_Tagset)==list:
        if len(initial_Tagset)>0:
            tagsets=initial_Tagset
            if type(initial_Tagset[0])==list:
                vase_tag=[]
                for ts in tagsets:
                    if type(ts)!=list:
                        raise ValueError("If Ts is list, type of all value is needed to list of tagset.")
                    vase_tag+=ts
            else:
                vasetagset
        else:
            raise ValueError("initail_Tagset list is NAN, you need to initial_Tagset length is higher than 0.")
    else:
        raise ValueError("initail_Tagset need to list:"+type(initial_Tagset))
    if type(Ts)==tuple or type(Ts)==list:
        if type(Ts)==tuple:
            if type(Ts[0])==int and type(Ts[1])==int:
                N=Ts[1]
                Ts=[Ts[0]]*N
            else:
                raise ValueError("tupple of Ts is need to (int,int):("+type(Ts[0])+","+type(Ts[1])+")")
        else:
            if len(Ts)>0:
                N=len(Ts)
            else:
                raise ValueError("Need to length of list of Ts is higher than 0.")
    else:
        raise ValueError("Ts need to tuple of (num of tag each post, num of post) or list of num of tag.:"+type(Ts))
    if type(alpha)==float or type(alpha)==list or type(alpha)==type(lambda x:x):
        if type(alpha)==list:
            if len(alpha)!=N:
                raise ValueError("different length between alpha and Ts:len(alpha)="+str(len(a))+",length of T="+str(len(N)))
            create_flag=3
            for a in alpha:
              if type(a)!=int and create_flag==3:
                create_flag=0
                break
            
        elif type(alpha)==float:
            alpha=[alpha]*N
        else:
            alpha=[alpha(t) for t in range(1,N+1)]
    else:
        if type(alpha)==int:
            alpha=[float(alpha)]*N
        else:
            raise ValueError("alpha need to list, float or function of novelty rate.:"+type(a))
    
    if type(new_tagset)==str or type(tagset)==list:
        pass
    else:
        raise ValueError("new_tagset need to str(something OK) or list of new tag.:"+type(new_tagset))
    if type(how_new_tag)==str:

        if create_flag==3:
            pass
        elif how_new_tag == "enumerate":
            create_flag=1
        elif how_new_tag == "str_enumerate":
            create_flag=2
        else:
            if type(new_tagset)!=list:
                print("Warning:You set non-'enumurate' value of how_new_tag, but you choice the new_tagset is str. The result is produced by only initial_Tagset.")
            create_flag=0
            
    else:
        raise ValueError("Type of how_new_tag is needed to str:"+type(how_new_tag))
    if type(from_num_new_tag)==int:
        while(from_num_new_tag in vase_tag):
            #print("Warning:There is number of from_num_new_tag in initilal_Tagset.Increaseing it")
            from_num_new_tag+=1
            
    else:
        raise ValueError("Type of from_num_new_tag is wrong.You should be int.")
    if type(fixed_T)!=bool:
      raise ValueError("Type of fixed_T is wrong.You should be it to True or False.")
    new_tag_id=from_num_new_tag

    def create_tag():
        nonlocal new_tagset
        nonlocal vase_tag
        nonlocal new_tag_id
        if type(new_tagset)==str:
            if create_flag==2:
                tag=str(new_tag_id)
            else:
                tag=new_tag_id
            new_tag_id+=1
        else:
            tag=new_tagset.pop(0)
        vase_tag.append(tag)
        return tag
    def PA():
        nonlocal vase_tag
        if len(vase_tag)==0:
          raise ValueError("There is no initial_Tagset. At least, you should set the initilal_Tagset=[[0]].")
        tag=random.choice(vase_tag)
        vase_tag.append(tag)
        return tag
    def YSP(a):
        
        if create_flag==0 and len(new_tagset)==0:
            a=0
        p=random.uniform(0, 1)
        if p<a:
            return create_tag()
        else:
            return PA()
    for i in range(N):
        tagset=[]
        b_vase_tag=vase_tag
        if create_flag==3:
          tagset=[PA() for _ in range(Ts[i]-alpha[i])]
          while fixed_T and len(tagset)!=len(set(tagset)):
            vase_tag=b_vase_tag
            tagset=[PA() for _ in range(Ts[i]-alpha[i])]
          tagset+=[create_tag() for _ in range(alpha[i])]
          
        else:
          tagset=[YSP(alpha[i]) for _ in range(Ts[i])]
          while fixed_T and len(tagset)!=len(set(tagset)):
            vase_tag=b_vase_tag
            tagset=[PA() for _ in range(Ts[i])]
        tagsets.append(tagset)
   
    return tagsets

In [6]:
YuleSimonProcess(Ts=(2,30))

[[0],
 [0, 1],
 [0, 1],
 [2, 1],
 [1, 0],
 [2, 1],
 [2, 1],
 [3, 1],
 [3, 2],
 [1, 0],
 [0, 2],
 [1, 0],
 [2, 1],
 [1, 2],
 [2, 1],
 [1, 2],
 [0, 2],
 [2, 1],
 [1, 4],
 [0, 1],
 [0, 4],
 [5, 1],
 [0, 1],
 [1, 0],
 [1, 0],
 [1, 6],
 [1, 5],
 [0, 1],
 [2, 1],
 [1, 2],
 [0, 1],
 [3, 0],
 [1, 7],
 [8, 5],
 [1, 0],
 [1, 5],
 [2, 1],
 [0, 2],
 [2, 3],
 [3, 4],
 [0, 1],
 [3, 1],
 [1, 1],
 [9, 2],
 [3, 1],
 [2, 1],
 [5, 3],
 [3, 10],
 [1, 10],
 [1, 2],
 [4, 1],
 [4, 0],
 [1, 5],
 [1, 0],
 [2, 1],
 [2, 1],
 [1, 4],
 [2, 1],
 [5, 1],
 [3, 1],
 [5, 1]]

In [8]:
YuleSimonProcess(alpha=0.3,Ts=(2,30),initial_Tagset=[[0,1]],fixed_T=True)

[[0, 1],
 [1, 0],
 [0, 1],
 [3, 1],
 [2, 1],
 [1, 4],
 [2, 1],
 [1, 0],
 [5, 1],
 [2, 5],
 [6, 1],
 [0, 1],
 [7, 8],
 [3, 9],
 [10, 0],
 [11, 12],
 [1, 13],
 [14, 1],
 [1, 10],
 [6, 14],
 [15, 16],
 [1, 17],
 [12, 15],
 [3, 18],
 [3, 1],
 [3, 17],
 [2, 0],
 [2, 1],
 [1, 8],
 [1, 0],
 [1, 0]]